In [22]:
import utils
import kaolin as kal
import torch
import numpy as np

#mesh = kal.rep.TriangleMesh.from_obj("model.obj") # Modelo de una esfera
#mesh = kal.rep.TriangleMesh.from_obj("cow.obj") # Modelo de vaca
mesh = kal.rep.TriangleMesh.from_obj("torus.obj")
normals = utils.compute_vertex_normals(mesh)
utils.visualize_normals(mesh, normals)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [2]:
import pointareas

pa, ca = pointareas.compute_pointareas(mesh)
pointareas.visualize_pointareas(mesh, pa)

f = open("/home/danniccs/rtsc-1.6/model_pointareas", "r")
tri_pa = [float(l) for l in f]
tri_pa_tens = torch.tensor(tri_pa)
pointareas.visualize_pointareas(mesh, tri_pa_tens)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.7761263…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.7761263…

In [23]:
# Calculate principal curvatures and curvature directions
import curvature

k1, k2, e1, e2 = curvature.compute_curvatures(mesh)

f = open("/home/danniccs/rtsc-1.6/k1curv", "r")
tri_k1 = [float(l) for l in f]
tri_k1_tens = torch.tensor(tri_k1)
f = open("/home/danniccs/rtsc-1.6/k2curv", "r")
tri_k2 = [float(l) for l in f]
tri_k2_tens = torch.tensor(tri_k2)

In [23]:
k1.mean()

tensor(1.1537)

In [24]:
tri_k1_tens.mean()

tensor(1.1382)

In [25]:
k2.mean()

tensor(0.9320)

In [26]:
tri_k2_tens.mean()

tensor(1.0770)

In [24]:
import meshplot as mp
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), k1.numpy(),
        shading={"colormap": "bwr"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [25]:
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), tri_k1_tens.numpy(),
        shading={"colormap": "bwr"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [26]:
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), k2.numpy(),
        shading={"colormap": "bwr"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [27]:
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), tri_k2_tens.numpy(),
        shading={"colormap": "bwr"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…